In [1]:
import tensorflow as tf
import tensorflow.keras as kr

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import math as mth
import random as random

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
#DECLARACION DE CLASES
#Los vectores de 8 elementos se llaman X (x mayuscula)
#Voy a generar un vector Y que es un vector de 1000 elemenos X. Con Y vamos a entrenar la red.

class pixel:
    #esta es la clase pixel, contiene tres canales de entre 0 y 255 cada uno
    #por una cuestion de entendimiento los canales se van a llamar R,G y B pero no tiene nada que ver con los colores en si,
    #son solo numeros
    def __init__(self,tiempo,frecR = 1,frecG = 2, frecB = 5):
        a = mth.floor(random.uniform(0,10)) #Aqui esta la aleatoriedad necesaria para entrenar bien
        b = mth.floor(random.uniform(0,10))
        self.R = mth.floor((127 * mth.sin(frecR*tiempo + a)) + 128 + b) #Aqui genero los tres canales del pixel
        self.G = mth.floor((127 * mth.sin(frecG*tiempo + a)) + 128 + b)
        self.B = mth.floor((127 * mth.sin(frecB*tiempo + a)) + 128 + b)
        #Esto se hace para que los valores sigan dentro del rango 0-255 :
        if self.R > 255: 
            self.R = 255
        if self.G > 255: 
            self.G = 255
        if self.B > 255: 
            self.B = 255
        if self.R < 0: 
            self.R = 0
        if self.G < 0: 
            self.G = 0
        if self.B < 0: 
            self.B = 0
    def getData(self): #Esto es para visualizar rapido en caso de que sea necesario
        print("R:",self.R, "G:",self.G, "B:", self.B)
#   def getNumbers(self):
 #      R = self.R
        #G = self.G
        #B = self.B
        
class pixels(list): #Esta es la clase de 8 elementos (cada elemento es un pixel)
    def fillWithPixels(self, cantElementos = 8):
        for x in range(cantElementos):
            #print("Iteracion numero ", x)
            self.append(pixel(x))

In [3]:
#prueba:
X = pixels()
X.fillWithPixels()
X[0].getData()
X[1].getData()
X[2].getData()
X[3].getData()
X[4].getData()
X[5].getData()
X[6].getData()
X[7].getData()

R: 215 G: 215 B: 215
R: 243 G: 145 B: 92
R: 4 G: 184 B: 150
R: 6 G: 253 B: 5
R: 189 G: 13 B: 51
R: 12 G: 64 B: 117
R: 92 G: 59 B: 2
R: 93 G: 22 B: 132


In [4]:
#GENERACION DEL DATASET DE ENTRENAMIENTO
#Quiero una matriz de 1000 filas. Cada fila es un vector de pixels. Es decir dataset1 y dataset2 son listas de 1000 elementos de pixels.
#Cada uno de ellos es un conjunto de 8 pixels

size = 1000
dataset1 = [] #Para entrenamiento
dataset2 = [] #Para testeo

for x in range(size):
    dataset1.append(pixels())
    dataset1[x].fillWithPixels()
    dataset2.append(pixels())
    dataset2[x].fillWithPixels()

    
#prueba de que anda todo bien hasta aqui:

dataset1[258][4].getData()

R: 181 G: 5 B: 43


In [12]:
#PREPARACION DEL DATASET PARA ENTRENAMIENTO
#Aqui voy a preparar el dataset para que sea compatible con la red y pueda ser usado para entrenarla:
#Seguramente tenga que convertir todo a tensores con las librerias scipy y/o numpy

#pruebo de hacerlo andar solo con los rojos (canal R):

dataset1_np = np.array(dataset1)
dataset2_np = np.array(dataset2)


#Estas cuatro asignaciones son para decirle a R-train y a r_test EL TAMAÑO QUE DEBEN TENER
#Debe haber una forma mas feliz de hacer esto pero no la descubri y si saco estas dos lineas me tira error
#------------------------------------inicio de solucion momentanea------------------------------------------
rx_train = dataset1_np
rx_test = dataset1_np
ry_train = np.zeros(1000)
ry_test = np.zeros(1000)
#----------------------------------------fin de solucion momentanea--------------------------------------------

k = 0

for i in range(1000):
    for j in range (8):
        rx_train[i][j] = dataset1_np[i][j].R
        rx_test[i][j] = dataset2_np[i][j].R
        if j == 7:
            ry_train[k] = rx_train[i][j] #La salida de la red neuronal es el ULTIMO elemento de un pixels (o sea T = 7) Eso es lo que quiero predecir
            ry_test[k] = rx_test[i][j]
            k = k + 1
            

#Y listo, con eso vamos a entrenar la red para UN SOLO CANAL. Mas adelante hay que hacerlo andar con los 3 canales.
#Para eso va a haber que hacer algo asi: (pero todavia no lo pude hacer andar)


#x_train = np.array(dataset1)
#y_train = x_train


#x_test = np.array(dataset2)
#y_test = x_test

In [21]:
#DEFINICION DE ARQUITECTURA DE RED NEURONAL

input_dim = 256 #segun documentacion de keras, debe ser el indice de entrada mas grande + 1
model = Sequential() #Creo la red neuronal
model.add(Embedding(input_dim, output_dim=256)) #Agrego la capa embedding. Suele ser la primer capa de un red en keras, siempre.
model.add(LSTM(128)) #Agrego una capa LSTM
model.add(Dropout(0.5)) #Agrego capa de Dropout
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='relu'))

In [22]:
#COMPILACION DE LA RED
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 256)         65536     
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 270,977
Trainable params: 270,977
Non-trainable params: 0
_________________________________________________________________


In [23]:
#ENTRENAMIENTO DE LA RED
#model.fit(r_train, r_train, batch_size=1000, epochs=100)
model.fit(rx_train, ry_train, epochs=100)
#model.save_weights("\first_try.h5")
score = model.evaluate(rx_test, ry_test)
score

Epoch 1/100
1000/1000 [==============================] - 3s 3ms/step - loss: -1100.6299 - acc: 0.0070
Epoch 2/100
1000/1000 [==============================] - 1s 701us/step - loss: -2440.6670 - acc: 0.0090
Epoch 3/100
1000/1000 [==============================] - 1s 709us/step - loss: -2447.9591 - acc: 0.0080
Epoch 4/100
1000/1000 [==============================] - 1s 712us/step - loss: -2450.6155 - acc: 0.0050
Epoch 5/100
1000/1000 [==============================] - 1s 705us/step - loss: -2450.6156 - acc: 0.0040
Epoch 6/100
1000/1000 [==============================] - 1s 715us/step - loss: -2450.6156 - acc: 0.0060
Epoch 7/100
1000/1000 [==============================] - 1s 715us/step - loss: -2448.2718 - acc: 0.0040
Epoch 8/100
1000/1000 [==============================] - 1s 712us/step - loss: -2450.6155 - acc: 0.0040
Epoch 9/100
1000/1000 [==============================] - 1s 701us/step - loss: -2450.6155 - acc: 0.0070
Epoch 10/100
1000/1000 [==============================] - 1s 730us

1000/1000 [==============================] - 1s 710us/step - loss: -2450.6155 - acc: 0.0100
Epoch 79/100
1000/1000 [==============================] - 1s 719us/step - loss: -2450.6155 - acc: 0.0100
Epoch 80/100
1000/1000 [==============================] - 1s 727us/step - loss: -2450.6155 - acc: 0.0080
Epoch 81/100
1000/1000 [==============================] - 1s 696us/step - loss: -2450.6155 - acc: 0.0080
Epoch 82/100
1000/1000 [==============================] - 1s 782us/step - loss: -2450.6155 - acc: 0.0040
Epoch 83/100
1000/1000 [==============================] - 1s 778us/step - loss: -2450.6155 - acc: 0.0040
Epoch 84/100
1000/1000 [==============================] - 1s 741us/step - loss: -2450.6155 - acc: 0.0050
Epoch 85/100
1000/1000 [==============================] - 1s 785us/step - loss: -2450.6155 - acc: 0.0090
Epoch 86/100
1000/1000 [==============================] - 1s 789us/step - loss: -2450.6155 - acc: 0.0060
Epoch 87/100
1000/1000 [==============================] - 1s 935us/s

[-2450.61553125, 0.007]

In [27]:
Z = [0,0,0,0,0,0,0,0]
Z[0] = X[0].R
Z[1] = X[1].R
Z[2] = X[2].R
Z[3] = X[3].R
Z[4] = X[4].R
Z[5] = X[5].R
Z[6] = X[6].R
Z[7] = X[7].R
Y = model.predict(Z)
Y.size

#No veo por que el size es 8. Seguire mañana

8

In [30]:
Y #Aca veo que claramente las salidas van entre 0 y 1. Y yo quiero que sean valores de entre 0 y 255. Quizas con multiplicar todo ya anda bien, quizas no

array([[0.19331807],
       [0.29150915],
       [0.20642994],
       [0.33677244],
       [0.26719856],
       [0.31464154],
       [0.289215  ],
       [0.28607416]], dtype=float32)

In [31]:
Z

[215, 243, 4, 6, 189, 12, 92, 93]